In [1]:
import open3d as o3d
import os
import numpy as np
from sklearn.neighbors import kneighbors_graph
import maxflow
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [11]:
datasetPath = "/PublicData/CadDataset/a1.0.0_10"
# part = "139661_105bb47a"#"140294_f659b875"
part = "140294_f659b875"
name = "assembly.obj"
file_path = os.path.join(datasetPath,part,name)

In [12]:
class DisjointSet:
    def __init__(self, n):
        self.parent = np.arange(n)
        self.size = np.ones(n, dtype=int)

    def find(self, u):
        while self.parent[u] != u:
            self.parent[u] = self.parent[self.parent[u]]  # 路径压缩
            u = self.parent[u]
        return u

    def union(self, u, v):
        u = self.find(u)
        v = self.find(v)
        if u == v:
            return False
        if self.size[u] < self.size[v]:
            u, v = v, u
        self.parent[v] = u
        self.size[u] += self.size[v]
        return True

def segment_pointcloud(points, k=8, c=0.5, min_size=20):
    """
    基于Felzenszwalb图分割思想的点云分割
    points: Nx3 numpy array 点云坐标
    k: 每个点邻居数量
    c: 合并阈值参数，越大区域越少
    min_size: 最小区域大小，太小的区域会被合并
    
    返回 labels: 每个点的分割类别
    """

    N = points.shape[0]

    # 1. 计算k近邻图边
    from sklearn.neighbors import NearestNeighbors
    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(points)
    distances, indices = nbrs.kneighbors(points)

    # 构建边列表：(点i, 点j, 权重)
    edges = []
    for i in range(N):
        for j in range(1, k+1):  # indices[i, 0]是自己，跳过
            neighbor = indices[i, j]
            dist = distances[i, j]
            edges.append((i, neighbor, dist))
    edges = np.array(edges, dtype=[('p1', int), ('p2', int), ('w', float)])

    # 2. 按权重排序边（距离越小越优先合并）
    edges.sort(order='w')

    # 3. 初始化并查集，每个点自己一个集合
    u = DisjointSet(N)

    # 4. 初始化阈值函数
    threshold = np.full(N, c)

    # 5. 依次遍历边，决定是否合并
    for edge in edges:
        a = u.find(edge['p1'])
        b = u.find(edge['p2'])
        if a != b:
            if edge['w'] <= threshold[a] and edge['w'] <= threshold[b]:
                merged = u.union(a, b)
                if merged:
                    a_new = u.find(a)
                    threshold[a_new] = edge['w'] + c / u.size[a_new]

    # 6. 合并小区域
    for edge in edges:
        a = u.find(edge['p1'])
        b = u.find(edge['p2'])
        if a != b:
            if u.size[a] < min_size or u.size[b] < min_size:
                u.union(a, b)

    # 7. 给每个点赋标签
    labels = np.zeros(N, dtype=int)
    label_map = {}
    label_id = 0
    for i in range(N):
        root = u.find(i)
        if root not in label_map:
            label_map[root] = label_id
            label_id += 1
        labels[i] = label_map[root]

    print(f"分割得到 {label_id} 个区域")
    return labels

In [13]:
# 读取网格
mesh = o3d.io.read_triangle_mesh(file_path)
if not mesh.has_vertices():
    raise RuntimeError("网格无顶点")
points = np.asarray(mesh.vertices)
print(f"读取网格顶点数: {len(points)}")

读取网格顶点数: 111342


In [14]:
labels = segment_pointcloud(points, k=20, c=0.5, min_size=30)

分割得到 2010 个区域


In [15]:
# 顶点染色
num_clusters = labels.max() + 1
colors = np.zeros((len(labels), 3))
cmap = plt.get_cmap("tab10")
for i in range(num_clusters):
    colors[labels == i] = cmap(i / max(1, num_clusters - 1))[:3]
mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

In [16]:
o3d.io.write_triangle_mesh("graphcut_alpha_segment_2004_k=20.ply",mesh)

True

In [ ]:
o3d.io.write_triangle_mesh("graphcut_alpha_segment2_k=16.ply",mesh)

True